# Assignment 1 Visione Artificiale,          Rosario Bongiorno

__________________________________________________________________________________________________

### Inizialmente vengono definite le varie funzioni che serviranno nello svolgimento dei vari punti.

In [1]:
import numpy as np
import sklearn
import cv2

def define_Points(n):
    n_corners = 24

    #3D points
    objp = np.zeros((n_corners,3), np.float32)
    
    #primi 6 punti 
    #first face
    objp[0,:] =[0,0,0]
    objp[1,:] =[2, 0, 2]
    #second face
    objp[2,:] =[0, 1, 1]
    objp[3,:] =[0, 4, 2]
    #top face
    objp[4,:] =[1, 1, 5]
    objp[5,:] =[8, 6, 5]
    
    #altri 6 punti
    #first face
    objp[6,:] =[5, 0, 2]
    objp[7,:] =[4, 0, 4]
    #second face
    objp[8,:] =[0, 2, 3]
    objp[9,:] =[0, 7, 0]
    #top face
    objp[10,:] =[5, 4, 5]
    objp[11,:] =[1, 5, 5]
    
    #altri 12 punti
    #first face
    objp[12,:] =[4,0,0]
    objp[13,:] =[8,0,0]
    objp[14,:] =[8,0,4]
    objp[15,:] =[1,0,4]
    
    #second face
    objp[16,:] =[0,4,0]
    objp[17,:] =[0,6,4]
    objp[18,:] =[0,5,3]
    objp[19,:] =[0,7,2]

    #top face
    objp[20,:] =[8,3,5]
    objp[21,:] =[7,1,5]
    objp[22,:] =[3,2,5]
    objp[23,:] =[4,6,5]

    #2D points
    imgp = np.zeros((n_corners,2), np.float32)

    #primi 6 punti 
    #first face
    imgp[0,:] = [548,661]
    imgp[1,:] = [633,559]
    
    #second face
    imgp[2,:] = [519,604]
    imgp[3,:] = [445,498]
    
    #top face
    imgp[4,:] = [579,420]
    imgp[5,:] = [673,291]
    
    
    #altri 6 punti
    #first face
    imgp[6,:] = [766,512]
    imgp[7,:] = [716,454]
    
    #second face
    imgp[8,:] = [496,498]
    imgp[9,:] = [383,511]
    
    #top face
    imgp[10,:] = [635,333]
    imgp[11,:] = [472,343]
    
    #altri 12 punti
    #first face
    imgp[12,:] = [691,608]
    imgp[13,:] = [806,557]
    imgp[14,:] = [837,417]
    imgp[15,:] = [607,487]
    
    #second face
    imgp[16,:] = [443,570]
    imgp[17,:] = [404,381]
    imgp[18,:] = [424,439]
    imgp[19,:] = [382,441]
    
    #top face
    imgp[20,:] = [751,331]
    imgp[21,:] = [783,368]
    imgp[22,:] = [624,382]
    imgp[23,:] = [552,311]
    
    
    
    return objp[:n,:], imgp[:n,:]

def show_points(imgp,img):
    # Let's show the selected points 
    #img = cv2.imread("img_square.jpeg")
    img_to_show = img.copy()
    for p in imgp:
        img_to_show = cv2.circle(img_to_show, p.astype(np.int32), 3, (255, 0, 0) )

    cv2.imshow("img", img_to_show)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def generate_P(imgp,objp):
    n_point, _=np.shape(imgp)
    
    A = np.zeros([n_point*2, 12])
    i = 0
    #costruisco la matrice A
    for point_imgp, point_objp in (zip(imgp, objp)):
        A[i, :] = np.array([point_objp[0], point_objp[1], point_objp[2], 1, 0, 0, 0, 0,
                            -(point_imgp[0] * point_objp[0]), -(point_imgp[0] * point_objp[1]),
                            -(point_imgp[0] * point_objp[2]), -point_imgp[0]
                            ])

        A[i + 1, :] = np.array([0, 0, 0, 0, point_objp[0], point_objp[1], point_objp[2], 1,
                                -(point_imgp[1] * point_objp[0]), -(point_imgp[1] * point_objp[1]),
                                -(point_imgp[1] * point_objp[2]), -point_imgp[1]
                                ])
        i += 2
    #applico SVD
    _, _, vh = np.linalg.svd(A)
    
    #prendo l'ultimo vettore di vh che contiene l'autovettore corrispondente all'autovalore più piccolo
    P = np.reshape(vh[-1, :], [3, 4])
    return P

def proietta(objp,P):
    n=np.array([1])
    imgp=np.zeros((len(objp),3))
    i=0
    for point in objp:
        
        point=np.concatenate([point,np.array([1])])
        imgp[i]=np.matmul(P,np.transpose(point))
        imgp[i]=imgp[i]/imgp[i][2]
        i=i+1
    return imgp[:,:2].astype(np.int32)
        
    


def draw_cube_points(P):
    points = []
    #frist face
    for x in range(9):
        for z in range(5):
            points.append([x, 0, z])

    for y in range(1, 8):
        for z in range(5):
            points.append([0, y, z])
        
    for x in range(9):
        for y in range(8):
            points.append([x, y, 5])
        
    points = np.asarray(points).astype(np.float32)

    projected_points = proietta(points,P)
    img = cv2.imread("img_square.jpeg")
    
    img_to_show = img.copy()
    for p in projected_points:
        img_to_show = cv2.circle(img_to_show, p.astype(np.int32), 3, (255, 0, 0) )
    
    cv2.imshow("result", img_to_show)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
def initialize_K(shape):
    # initial guess of K
    K = np.zeros((3,3))

    K[0,0]=500
    K[1,1]=500
    K[2,2]=1
    K[0,2]=shape[0]/2
    K[1,2]=shape[1]/2
    return K
   


In [2]:
img = cv2.imread("img_square.jpeg")
objp, imgp = define_Points(6)
show_points(imgp,img)

objp, imgp = define_Points(12)
show_points(imgp,img)

objp, imgp = define_Points(24)
show_points(imgp,img)

# Punti 1 e 2

### Vengono calcolati i parametri intrinseci ed estrinseci della camera, implementando l'algoritmo DLT.
### L'immagine e il sistema di riferimento usati sono i seguenti.

![](img_square.jpeg)

### La funzione dlt(imgp, objp) implementa l'algoritmo dlt

In [3]:

def dlt(imgp, objp):
    
    P= generate_P(imgp,objp)
   
    
    P_33=np.reshape(P[:3,:3],[3,3])
    
    #inverto la matrice P per applicare la fattorizzazione QR
    P_inv=np.linalg.inv(P_33)
    
    #applico QR
    R_inv,K_inv=np.linalg.qr(P_inv)
    
    #la fattorizzazione mi restituisce la matrici inverse K ed R, quindi procedo invertendole
    K=np.linalg.inv(K_inv)
    R=np.linalg.inv(R_inv)
    
    #moltiplico la matrice K per la matrice di riflessione Rz per aggiustare i valori ottenuti
    Rz= np.array([[1,0,0],[0,-1,0],[0,0,1]])
    K=np.matmul(K,Rz)
    
    #applico la normalizzazione per far si che l'elemento di posto (2,2) sia 1
    scala=K[2][2]
    K=K/scala
    
    #infine calcolo T=Kinv*P4
    K_inv=np.linalg.inv(K)
    P_4=np.reshape(P[:,-1],[3,1])
    T=np.matmul(K_inv,P_4)
    
    return P, R, K, T


In [4]:
import pandas as pd

data = [
    #6
    [0,0,0,0,0],
    #12
    [0,0,0,0,0],
    #24
    [0,0,0,0,0]]


for i in range(3):
    if i==0:
        n_points=6
    elif i==1:
        n_points=12
    else:
        n_points =24

    #Calcolo K utilizzando il metodo che ho implementato
    objp, imgp = define_Points(n_points)
    P, R, K,T=dlt(imgp,objp)
    draw_cube_points(P)
    
    #Calcolo K utilizzando il metodo di openCV
    img = cv2.imread("img_square.jpeg")
    K_cv2 = initialize_K(img.shape)
    _, K_cv2, _, _, _ = cv2.calibrateCamera([objp], [imgp], img.shape[0:2], K_cv2, None, flags=cv2.CALIB_USE_INTRINSIC_GUESS|cv2.CALIB_FIX_S1_S2_S3_S4| cv2.CALIB_ZERO_TANGENT_DIST| cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K3 )

    print(" ")
    print('------------------------------'+str(n_points)+' Punti:')
    print('P')
    print(P)
    print(" ")
    print("Parametri Intrinseci:")
    print(" ")
    print('K')
    print(K)
    print(" ")
    print("Parametri Estrinseci:")
    print(" ")
    print('R')
    print(R)
    print(" ")
    print('T')
    print(T)
    print(" ")
   
   
    #Calcolo le differenze assolute e le salvo il una tabella
    differenze=np.absolute(K_cv2-K)
    data[i][0]=differenze[0][0]#f
    data[i][1]=differenze[1][1]#af
    data[i][2]=differenze[0][1]#s
    data[i][3]=differenze[0][2]#cx
    data[i][4]=differenze[1][2]#af
   



 
------------------------------6 Punti:
P
[[ 7.77566794e-02 -6.65062099e-03 -3.77731755e-02  6.32701464e-01]
 [ 1.19372431e-02 -2.74480693e-03 -8.01220160e-02  7.65245576e-01]
 [ 5.78537607e-05  4.59038570e-05 -7.43031831e-05  1.15788195e-03]]
 
Parametri Intrinseci:
 
K
[[526.3050091  -33.84156391 637.79284914]
 [ -0.         495.8739576  593.87957153]
 [ -0.          -0.           1.        ]]
 
Parametri Estrinseci:
 
R
[[-0.71327408  0.68874907 -0.12986455]
 [-0.43159487 -0.57760819 -0.69288862]
 [-0.55223722 -0.43817062  0.70925352]]
 
T
[[-0.00019094]
 [ 0.0001565 ]
 [ 0.00115788]]
 
 
------------------------------12 Punti:
P
[[ 5.98560732e-02 -9.68597791e-03 -4.85928658e-03  6.32528729e-01]
 [-3.88998605e-03 -2.48303857e-03 -5.69272936e-02  7.70028494e-01]
 [ 2.06403301e-05  4.62745022e-05 -1.47938331e-05  1.15249449e-03]]
 
Parametri Intrinseci:
 
K
[[ 1.10948436e+03 -4.47132000e+01  3.08348148e+02]
 [-0.00000000e+00  1.05681141e+03  2.32208213e+02]
 [-0.00000000e+00 -0.00000

### Nella seguente tabella si evidenziano le differenze assolute tra i valori della matrice K calcolata con openCV e quelli della matrice K calcolata con l'algolritmo dlt precedentemente implementeto, per 6, 12 e 24 punti

In [5]:
df = pd.DataFrame(data,columns =["fx", "fy","s", "cx","cy"],index =["6 punti", "12 punti","24 punti"])

display(df)

,fx,fy,s,cx,cy
6 punti,221.938609,203.910782,33.841564,101.673172,68.179739
12 punti,299.472989,466.735895,44.713200,221.482444,568.871755
24 punti,263.434827,241.922519,10.860516,39.889233,83.662072



______________________________________________________________________________________

## Punto 3

### Per risolvere il punto 3 vengono calcolati i pixel corrispondenti al pavimento. Il sistema di riferimento e i punti usati sono i seguenti.

![](floor1.png)

In [6]:

def define_floor_Points():
    n_point = 13

    #3D points
    floor3d = np.zeros((n_point,3), np.float32)

    floor3d[1,:] =[51,73, 0]
    floor3d[2,:] =[-128,-93 , 0]
    floor3d[3,:] =[-305,-92.5 , 0]
    floor3d[4,:] =[-305, -53, 0]
    floor3d[5,:] =[-305, 0, 0]
    floor3d[6,:] =[-305, 123, 0]
    floor3d[7,:] =[-128, -211, 0]
    floor3d[8,:] =[92, 60, 81]
    floor3d[9,:] =[-38, 164, 30]
    floor3d[10,:] =[-369, 37, 78]
    floor3d[11,:] =[-277, -149, 89]
    floor3d[12,:] =[-338, -28, 76.5]

    #2D points
    floor2d = np.zeros((n_point,2), np.float32)

    floor2d[0,:] = [721,551]
    floor2d[1,:] = [470,633]
    floor2d[2,:] = [947,379]
    floor2d[3,:] = [899,228]
    floor2d[4,:] = [831,221]
    floor2d[5,:] = [737,216]
    floor2d[6,:] = [522,207]
    floor2d[7,:] = [1169,392]
    floor2d[8,:] = [461,501]
    floor2d[9,:] = [274,320]
    floor2d[10,:] = [688,58]
    floor2d[11,:] = [1022,95]
    floor2d[12,:] = [794,77]
    
    return floor3d, floor2d


### Per valutare la qualità della riproiezione dei punti viene utilizzata la cross validation, calcolando l'MSE e la deviazione standard su 4 insiemi

In [7]:
floor3d,floor2d=define_floor_Points()
n_point, _=np.shape(floor3d)

#effettuo uno shuffle dei punti
floor3d, floor2d=sklearn.utils.shuffle(floor3d,floor2d)


P_matrix = [0,0,0,0]
for i in range(4):
    print("Run "+str(i+1))
    
    #definisco gli insiemi
    validationset3d=floor3d[i*3:(i*3)+3,:]
    trainingset3d=np.concatenate((floor3d[:i*3,:], floor3d[(i*3)+3:,:]))
    
    validationset2d=floor2d[i*3:(i*3)+3,:]
    trainingset2d=np.concatenate((floor2d[:i*3,:], floor2d[(i*3)+3:,:]))
    
    #per ogni run genero la matrice P
    P=generate_P(trainingset2d, trainingset3d)
    P_matrix[i]=P;
    
    #proietto i punti di validation utilizzando la matrice P ricavata precedentemente con il training set
    vset2d_proiettati= proietta(validationset3d, P)
    img = cv2.imread("floor.jpeg")
    show_points(vset2d_proiettati,img)
    #calcolo l'errore quadratco medio
    mean_error = (cv2.norm(vset2d_proiettati.astype(np.int32), validationset2d.astype(np.int32), normType=cv2.NORM_L2)**2)/len(validationset2d)
    #calcolo la deviazione standard
    std=np.std(vset2d_proiettati.astype(np.int32)-validationset2d.astype(np.int32))
    print( "mean error: {}".format(mean_error) )
    print( "std: {}".format(std) )
    
    
    

Run 1
mean error: 26.333333333333332
std: 3.593976442141304
Run 2
mean error: 2099.0000000000005
std: 29.134267719569607
Run 3
mean error: 245.33333333333334
std: 10.327955589886445
Run 4
mean error: 1473.6666666666667
std: 24.741440719749708


### Infine viene disegnato il pavimento proiettando i punti corrispondenti

In [9]:
def draw_ground(P,img):
    xg = np.arange(-305, 51, 10) 
    yg = np.arange(-211, 164, 10) 
    xx, yy = np.meshgrid(xg, yg)

    dim = xx.shape[0]*xx.shape[1]
    points = np.zeros((dim,3), np.float32)

    xx = xx.reshape(dim)
    yy = yy.reshape(dim)

    points[:,0] = xx 
    points[:,1] = yy 
    points[:,2] = np.zeros((dim))

    ground= proietta(points, P)
    ground = np.squeeze(ground).astype(np.int32)

    img_to_show_res = img.copy()
    for p in ground:
        img_to_show_res = cv2.circle(img_to_show_res, p, 3, (255, 0, 0) )

    cv2.imshow("ground", img_to_show_res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#disegno il pavimento utilizzando le matrici P generate nella fase di training
for i in range(4):  
    img = cv2.imread("floor.jpeg")    
    draw_ground(P_matrix[i],img)